In [13]:
from pyomo.opt import SolverFactory
import pyomo.core as pyo
import pyomo.environ
import numpy as np
import pandas as pd
import gurobipy as gp

In [3]:
# Load the data
file_path = "/Users/oscar/Documents/GitHub/Risk_Model_Research/ncd_milp/sim_milp/"
file_name = "sim_50_3_2_7_1_1_data.csv"  # Replace with your file name
df = pd.read_csv(file_path + file_name) 

# Preprocessing the data
#df = df.iloc[0:10,]
y = df.iloc[:, 0].values
X = df.iloc[:, 1:].values

# Parameters
n, p = X.shape
M = 1000
SK_pool = np.linspace(-5 * p, 5 * p + 1,10 * p + 2,dtype=int)
PI = np.linspace(0, 1, 100)[1:-1]  # Exclude 0 and 1


In [10]:
# Define the pyomo model
model = pyo.ConcreteModel()

# Variables
model.beta = pyo.Var(range(p), within=pyo.Integers, bounds=(-5, 5))
model.s = pyo.Var(range(n), within=pyo.Integers, bounds=(-10, 10))
model.z_ik = pyo.Var(range(n), range(len(SK_pool)), within=pyo.Binary)
model.p_ik = pyo.Var(range(n), range(len(SK_pool)), within=pyo.NonNegativeReals, bounds=(0.0001, 0.9999))
model.p_k = pyo.Var(range(len(SK_pool)), within=pyo.NonNegativeReals, bounds=(0.0001, 0.9999))


# Constraints
def score_constraint_rule(model, i):
    return sum(model.beta[j] * X[i, j] for j in range(p)) == model.s[i]
model.score_constraint = pyo.Constraint(range(n), rule=score_constraint_rule)

def z_ik_constraint_rule(model, i):
    return sum(model.z_ik[i, k] for k in range(len(SK_pool))) == 1
model.z_ik_constraint = pyo.Constraint(range(n), rule=z_ik_constraint_rule)

def s_z_ik_constraint_rule_1(model, i, k):
    return model.s[i] - k - M * (1 - model.z_ik[i, k]) <= 0
model.s_z_ik_constraint_1 = pyo.Constraint(range(n), range(len(SK_pool)), rule=s_z_ik_constraint_rule_1)

def s_z_ik_constraint_rule_2(model, i, k):
    return model.s[i] - k + M * (1 - model.z_ik[i, k]) >= 0
model.s_z_ik_constraint_2 = pyo.Constraint(range(n), range(len(SK_pool)), rule=s_z_ik_constraint_rule_2)

def p_ik_constraint_rule_1(model, i, k):
    return model.p_ik[i, k] - model.p_k[k] <= M * (1 - model.z_ik[i, k])
model.p_ik_constraint_1 = pyo.Constraint(range(n), range(len(SK_pool)), rule=p_ik_constraint_rule_1)

def p_ik_constraint_rule_2(model, i, k):
    return model.p_ik[i, k] - model.p_k[k] >= -M * (1 - model.z_ik[i, k])
model.p_ik_constraint_2 = pyo.Constraint(range(n), range(len(SK_pool)), rule=p_ik_constraint_rule_2)

# Objective Function
def objective_rule(model):
    #return 0
    return -sum(y[i] * pyo.log(model.p_ik[i, k]) + (1 - y[i]) * pyo.log(1 - model.p_ik[i, k])
                for i in range(n) for k in range(len(SK_pool)))
model.objective = pyo.Objective(rule=objective_rule)

In [ ]:
# Solve the model gurobi
opt = SolverFactory('gurobi')
opt.options['MIPGap'] = 0.01
opt.options['TimeLimit'] = 60
opt.options['Threads'] = 1
results = opt.solve(model)

In [18]:
# Solve the model gurobi bonmin
solver = SolverFactory('couenne')
#solver.options['bonmin.time_limit'] = 3600
#solver.options['bonmin.cutoff_decr'] = 0.01  # or another small value to adjust the cutoff incrementally
#solver.options['bonmin.allowable_gap'] = 1E-2  # Adjusts the gap considered acceptable for stopping.
solver.solve(model, tee=True).write()

for solver asl. File with name=couenne either does not exist or it is not
executable. To skip this validation, call set_executable with validate=False.
Traceback (most recent call last):
  File "/Users/oscar/anaconda3/lib/python3.11/site-packages/pyomo/opt/base/solvers.py", line 162, in __call__
    opt = self._cls[_implicit_solvers[mode]](**kwds)
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/oscar/anaconda3/lib/python3.11/site-packages/pyomo/solvers/plugins/solvers/ASL.py", line 45, in __init__
    SystemCallSolver.__init__(self, **kwds)
  File "/Users/oscar/anaconda3/lib/python3.11/site-packages/pyomo/opt/solver/shellcmd.py", line 65, in __init__
    self.set_executable(name=executable, validate=validate)
  File "/Users/oscar/anaconda3/lib/python3.11/site-packages/pyomo/opt/solver/shellcmd.py", line 114, in set_executable
    raise ValueError(
ValueError: Failed to set executable for solver asl. File with name=couenne either does not exist or it is not executabl

RuntimeError: Attempting to use an unavailable solver.

The SolverFactory was unable to create the solver "couenne"
and returned an UnknownSolver object.  This error is raised at the point
where the UnknownSolver object was used as if it were valid (by calling
method "solve").

The original solver was created with the following parameters:
	executable: couenne
	type: couenne
	_args: ()
	options: {}

In [16]:
# Solve the model gurobi bonmin
solver = SolverFactory('cbc')
#solver.options['bonmin.time_limit'] = 3600
#solver.options['bonmin.cutoff_decr'] = 0.01  # or another small value to adjust the cutoff incrementally
#solver.options['bonmin.allowable_gap'] = 1E-2  # Adjusts the gap considered acceptable for stopping.
solver.solve(model, tee=True).write()

ValueError: Model objective (objective) contains nonlinear terms that cannot be written to LP format